## Observations 
available:
| Dataset    | variables   | ESMValTool-find |
|:------------|:-------------|:--------------------|
|ERA-Interim | sst(tos, Omon)(ts, Amon), pr, taux(tauu) |OBS6, reanaly, tier3 |
|NOAA-CIRES-20CR-V2     |prw,  |OBS6, Tier2
|HadISST     |tos/ts,   |OBS, Tier2
|NOAA-ERSSTv5|tos      |OBS6, Tier2
|NCEP-NCAR-R1|ts, pr   |OBS6, Tier2
|NCEP        |pr     |OBS, Tier2
|GPCP-SG     |pr,    |OBS, Tier2

Need to get:
- TropFlux - sst, taux (https://incois.gov.in/tropflux/DataHome.jsp)
- CMAP - pr (https://psl.noaa.gov/data/gridded/data.cmap.html)

In [1]:
from esmvalcore.config import CFG
from esmvalcore.dataset import Dataset

CFG.load_from_file('/home/189/fc6164/esmValTool/config-fc-copy.yml')

obs = Dataset(dataset= 'ERA-Interim', project= 'OBS6', 
              type= 'reanaly', version='1', 
              timerange='1979/2018', tier='3', 
              mip='Omon', short_name='ts')

print(len(obs.files))
list(obs.from_files())
# obs.find_files(), obs.files

0


[Dataset:
 {'dataset': 'ERA-Interim',
  'project': 'OBS6',
  'mip': 'Omon',
  'short_name': 'ts',
  'frequency': 'mon',
  'long_name': 'Surface Temperature',
  'modeling_realm': ['atmos'],
  'original_short_name': 'ts',
  'standard_name': 'surface_temperature',
  'tier': '3',
  'timerange': '1979/2018',
  'type': 'reanaly',
  'units': 'K',
  'version': '1'}
 session: 'session-ef9ba2fe-c08f-481a-a0ac-67187eddfb4a_20241001_043535']

In [4]:
from esmvalcore.config import CFG
CFG.load_from_file('/home/189/fc6164/esmValTool/config-fc-copy.yml')
from esmvalcore.dataset import Dataset
obs_sst = Dataset(dataset= 'NOAA-ERSSTv5', 
                 project= 'OBS6', 
                 type= '*', 
                 version='*' , 
                 timerange = '1970/2018', 
                 tier='*', 
                 mip='*', 
                 short_name='tos')

obs_sst.find_files()
obs_sst.files

# list(obs_sst.from_files())

[LocalFile('/g/data/kj13/datasets/esmvaltool/obsdata-v2/Tier2/NOAA-ERSSTv5/OBS6_NOAA-ERSSTv5_reanaly_v5_Omon_tos_185401-200712.nc'),
 LocalFile('/g/data/kj13/datasets/esmvaltool/obsdata-v2/Tier2/NOAA-ERSSTv5/OBS6_NOAA-ERSSTv5_reanaly_v5_Omon_tos_200801-202001.nc')]